# Intra-Annual Extremes


In [1]:
%run ../setup.ipynb
%run ../plotting_functions.ipynb
data_dir = Path('../../data' )
output_dir = Path('../../output') 

# We're going to use plotly here, so we need to import it
import plotly.io as pio
import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go

# check to make sure that data_dir/rsl_daily_hawaii.nc exists, if not, make warning to run datawrangling notebook
if not (data_dir / 'rsl_hawaii.nc').exists():
    print('rsl_hawaii.nc not found in '+ str(data_dir) +  '. Please run the data wrangling notebook first')
else:
    print('rsl_hawaii.nc found in '+ str(data_dir) +  '. Proceed.')

rsl_hawaii.nc found in ../../data. Proceed.


In [2]:
# open the dataset
# Load dataset with Dask (assuming your dataset is in NetCDF format)
ds = xr.open_dataset(data_dir / 'rsl_hawaii.nc', chunks={'time': 1000})



ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'scipy']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [3]:
# Separate the sea_level variable
sea_level = ds['sea_level']



# resample to daily min, max, and mean
daily_mean = sea_level.resample(time='1D').mean()
daily_max = sea_level.resample(time='1D').max()
daily_min = sea_level.resample(time='1D').min()



# # Group by 'dayofyear' and calculate the mean, max, and min for each day of the year
# avg_daily = daily_mean.groupby('dayofyear').mean(dim='time')
# record_high = sea_level.groupby('dayofyear').max(dim='time')
# record_low = sea_level.groupby('dayofyear').min(dim='time')





In [4]:
daily_mean

<xarray.DataArray 'sea_level' (record_id: 11, time: 43495)> Size: 2MB
dask.array<groupby_nanmean, shape=(11, 43495), dtype=float32, chunksize=(11, 83), chunktype=numpy.ndarray>
Coordinates:
  * record_id  (record_id) int16 22B 140 500 520 570 580 ... 610 5470 5480 5520
  * time       (time) datetime64[ns] 348kB 1905-01-01 1905-01-02 ... 2024-01-31
Attributes:
    long_name:  relative sea level
    units:      millimeters
    source:     in situ tide gauge water level observations
    platform:   station_name, station_country, station_country_code, uhslc_id...

In [5]:
# Add a 'dayofyear' coordinate to sea_level only
def add_dayofyear(ds):
    return ds.assign_coords(dayofyear=ds['time'].dt.dayofyear)
daily_mean = add_dayofyear(daily_mean)
daily_max = add_dayofyear(daily_max)
daily_min = add_dayofyear(daily_min)

In [16]:
avg_daily = daily_mean.sel('time'=slice(1983,2001)).groupby('dayofyear').mean(dim='time') #CHECK IF THIS IS RIGHT, SOMETHING IS WRONG
avg_high = daily_max.groupby('dayofyear').mean(dim='time')
avg_low = daily_min.groupby('dayofyear').mean(dim='time')

record_low = daily_min.groupby('dayofyear').min(dim='time')
record_high = daily_max.groupby('dayofyear').max(dim='time')


In [18]:
avg_daily

<xarray.DataArray 'sea_level' (record_id: 11, dayofyear: 366)> Size: 16kB
dask.array<getitem, shape=(11, 366), dtype=float32, chunksize=(11, 33), chunktype=numpy.ndarray>
Coordinates:
  * record_id  (record_id) int16 22B 140 500 520 570 580 ... 610 5470 5480 5520
  * dayofyear  (dayofyear) int64 3kB 1 2 3 4 5 6 7 ... 361 362 363 364 365 366
Attributes:
    long_name:  relative sea level
    units:      millimeters
    source:     in situ tide gauge water level observations
    platform:   station_name, station_country, station_country_code, uhslc_id...

In [21]:
def make_plotlyFigure(mean, max, min, record_high,record_low, rid,ds):
    figly = go.Figure()
    
    # Select data for the given record_id
    mean_rid = mean.isel(record_id=rid)
    max_rid = max.isel(record_id=rid)
    min_rid = min.isel(record_id=rid)
    record_high_rid = record_high.isel(record_id=rid)
    record_low_rid = record_low.isel(record_id=rid)

    x = mean_rid.dayofyear.values

         # Add traces
    figly.add_trace(go.Scatter(x=x, y=max_rid.values, mode='lines', line_color='rgba(0,176,246,0.2)', name='Average High'))
    figly.add_trace(go.Scatter(x=x, y=min_rid.values, mode='lines', line_color='rgba(0,176,246,0.2)', fill='tonexty', fillcolor='rgba(0,176,246,0.2)', name='Average High/Low'))
    figly.add_trace(go.Scatter(x=x, y=mean_rid.values, mode='lines', line_color='rgba(0,176,246,1)', name='Average Daily'))
    figly.add_trace(go.Scatter(x=x, y=record_high_rid.values, mode='lines', line_color='rgba(0,176,246,0.2)', name='Record High'))
    figly.add_trace(go.Scatter(x=x, y=record_low_rid.values, mode='lines', line_color='rgba(0,176,246,0.2)', fill='tonexty', fillcolor='rgba(0,176,255,0.2)', name='Record High/Low'))
    
    # Update layout
    figly.update_layout(
        title=str(ds['station_name'].isel(record_id=rid).values),
        xaxis_title='Time',
        yaxis_title=ds['sea_level'].attrs['long_name'] + ' [' + ds['sea_level'].attrs['units'] + ']',
        showlegend=True,
        plot_bgcolor='white',
        xaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='rgb(200,200,200)'),
        yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='rgb(200,200,200)'),
        autosize=True,
        legend=dict(orientation='h', y=1.1, x=0.5, xanchor='center', yanchor='top'),
        title_x=0.5,
        title_font_size=24,
        title_font_family='Arial',
        title_font_color='black',
        title_font=dict(size=20, color='black')
    )

    figly.show()

    return figly

In [22]:
#plot the montly max/mean and min values of Honolulu
figly = make_plotlyFigure(avg_daily, avg_high, avg_low, record_high, record_low, 3,ds)

In [ ]:
import plotly.graph_objects as go

rid = 3

fig = go.Figure()

# plot the monthly max, min, and mean values
fig.add_trace(go.Scatter(x=ds_monthly_max.time, y=ds_monthly_max.sea_level.isel(record_id=rid), mode='lines', name='Max'))
fig.add_trace(go.Scatter(x=ds_monthly_min.time, y=ds_monthly_min.sea_level.isel(record_id=rid), mode='lines', name='Min'))
fig.add_trace(go.Scatter(x=ds_monthly_mean.time, y=ds_monthly_mean.sea_level.isel(record_id=rid), mode='lines', name='Mean'))

fig.update_layout(title='Monthly Max, Min, and Mean Sea Level in Hawaii',
                     xaxis_title='Year',
                     yaxis_title='Sea Level (mm)')
fig.show()

